In [5]:
import numpy as np

import pandas as pd

## 一、长宽表的变形
什么是长表？什么是宽表？这个概念是对于某一个特征而言的。例如：一个表中**把性别Gender单独存储在某一个列中，那么它就是关于性别的长表**；如果**把性别的值作为列名的一部分，列中的元素是某一其他的相关特征数值，那么这个表就是关于性别的宽表**。下面的两张表就分别是关于性别的长表和宽表：

In [6]:
pd.DataFrame({'Gender':['F','F','M','M'],
              'Height':[163, 160, 175, 180]})

,Gender,Height
0,F,163
1,F,160
2,M,175
3,M,180


In [7]:
pd.DataFrame({'Height: F':[163, 160],
              'Height: M':[175, 180]})

,Height: F,Height: M
0,163,175
1,160,180


显然这两张表从信息上是完全等价的，它们包含相同的身高统计数值，只是这些数值的呈现方式不同，而其呈现方式主要又与性别一列选择的布局模式有关，**即到底是以 long 的状态存储还是以 wide 的状态存储。因此， pandas 针对此类长宽表的变形操作设计了一些有关的变形函数。**

### 1. pivot
pivot 是一种典型的长表变宽表的函数，首先来看一个例子：下表存储了张三和李四的语文和数学分数，现在想要把语文和数学分数作为列来展示。

In [1]:
df = pd.DataFrame({'Class':[1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Math','Chinese','Math'],
                   'Grade':[80,75,90,85]})
df

NameError: name 'pd' is not defined

对于一个基本的长变宽操作而言，最重要的有**三个要素**，分别是**变形后的行索引、需要转到列索引的列，以及这些列和行索引对应的数值**，它们分别对应了 pivot 方法中的 index, columns, values 参数。新生成表的列索引是 columns 对应列的 unique 值，而新表的行索引是 index 对应列的 unique 值，而 values 对应了想要展示的数值列。

In [12]:

df.pivot(index='Name',columns='Subject',values='Grade')

Subject,Chinese,Math
Name,,
San Zhang,80,75
Si Li,90,85


![](https://inter.joyfulpandas.datawhale.club/_images/ch5_pivot.png)

利用 pivot 进行变形操作需要满足唯一性的要求，即由于在新表中的行列索引对应了唯一的 value ，因此**原表中的 index 和 columns 对应两个列的行组合必须唯一**。例如，现在把原表中第二行张三的数学改为语文就会报错，这是由于 Name 与 Subject 的组合中两次出现 ("San Zhang", "Chinese") ，从而最后不能够确定到底变形后应该是填写80分还是75分。

In [13]:
df.loc[1,'Subject'] = 'Chinese'

try:
 df.pivot(index='Name', columns='Subject', values='Grade')
except Exception as e:
   Err_Msg = e


Err_Msg
# ValueError('Index contains duplicate entries, cannot reshape')

ValueError('Index contains duplicate entries, cannot reshape')

pandas 从 1.1.0 开始， **pivot 相关的三个参数允许被设置为列表，这也意味着会返回多级索引**。这里构造一个相应的例子来说明如何使用：下表中六列分别为班级、姓名、测试类型（期中考试和期末考试）、科目、成绩、排名。

In [15]:
df = pd.DataFrame({'Class':[1, 1, 2, 2, 1, 1, 2, 2],
                  'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li',
                           'San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                  'Examination': ['Mid', 'Final', 'Mid', 'Final',
                                 'Mid', 'Final', 'Mid', 'Final'],
                  'Subject':['Chinese', 'Chinese', 'Chinese', 'Chinese',
                              'Math', 'Math', 'Math', 'Math'],
                  'Grade':[80, 75, 85, 65, 90, 85, 92, 88],
                  'rank':[10, 15, 21, 15, 20, 7, 6, 2]})
df

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


现在想要把测试类型和科目联合组成的四个类别（期中语文、期末语文、期中数学、期末数学）转到列索引，并且同时统计成绩和排名：

In [18]:
pivot_multi = df.pivot(index=['Class','Name'],
                columns=['Subject','Examination'],
                values=['Grade','rank'])
pivot_multi



Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2

In [19]:
print(pivot_multi.to_markdown())

|                  |   ('Grade', 'Chinese', 'Mid') |   ('Grade', 'Chinese', 'Final') |   ('Grade', 'Math', 'Mid') |   ('Grade', 'Math', 'Final') |   ('rank', 'Chinese', 'Mid') |   ('rank', 'Chinese', 'Final') |   ('rank', 'Math', 'Mid') |   ('rank', 'Math', 'Final') |
|:-----------------|------------------------------:|--------------------------------:|---------------------------:|-----------------------------:|-----------------------------:|-------------------------------:|--------------------------:|----------------------------:|
| (1, 'San Zhang') |                            80 |                              75 |                         90 |                           85 |                           10 |                             15 |                        20 |                           7 |
| (2, 'Si Li')     |                            85 |                              65 |                         92 |                           88 |                           21 |                

In [21]:
# 将表格转换为字符串
print(pivot_multi.to_string())

                  Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2


In [22]:
# 将表格转换为 LaTeX 字符串
print(pivot_multi.to_latex())

\begin{tabular}{llrrrrrrrr}
\toprule
 &  & \multicolumn{4}{r}{Grade} & \multicolumn{4}{r}{rank} \\
 & Subject & \multicolumn{2}{r}{Chinese} & \multicolumn{2}{r}{Math} & \multicolumn{2}{r}{Chinese} & \multicolumn{2}{r}{Math} \\
 & Examination & Mid & Final & Mid & Final & Mid & Final & Mid & Final \\
Class & Name &  &  &  &  &  &  &  &  \\
\midrule
1 & San Zhang & 80 & 75 & 90 & 85 & 10 & 15 & 20 & 7 \\
\cline{1-10}
2 & Si Li & 85 & 65 & 92 & 88 & 21 & 15 & 6 & 2 \\
\cline{1-10}
\bottomrule
\end{tabular}



根据唯一性原则，新表的行索引等价于对 index 中的多列使用 drop_duplicates ，而列索引的长度为 values 中的元素个数乘以 columns 的唯一组合数量（与 index 类似） 。从下面的示意图中能够比较容易地理解相应的操作：

<img src="https://inter.joyfulpandas.datawhale.club/_images/ch5_mulpivot.png" width="70%" >

### 2. pivot_table
**pivot 的使用依赖于唯一性条件，那如果不满足唯一性条件，那么必须通过聚合操作使得相同行列组合对应的多个值变为一个值**。例如，张三和李四都参加了两次语文考试和数学考试，按照学院规定，最后的成绩是两次考试分数的平均值，此时就无法通过 pivot 函数来完成。

In [23]:
df = pd.DataFrame({'Name':['San Zhang', 'San Zhang',
                           'San Zhang', 'San Zhang',
                           'Si Li', 'Si Li', 'Si Li', 'Si Li'],
                  'Subject':['Chinese', 'Chinese', 'Math', 'Math',
                              'Chinese', 'Chinese', 'Math', 'Math'],
                  'Grade':[80, 90, 100, 90, 70, 80, 85, 95]})


df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


pandas 中提供了 pivot_table 来实现，**其中的 aggfunc 参数就是使用的聚合函数**。上述场景可以如下写出：

In [24]:
df.pivot_table(index='Name',columns='Subject',values='Grade',aggfunc='mean')

Subject,Chinese,Math
Name,,
San Zhang,85.0,95.0
Si Li,75.0,90.0


这里传入 aggfunc 包含了上一章中介绍的所有合法聚合字符串，**此外还可以传入以序列为输入标量为输出的聚合函数来实现自定义操作**，上述功能可以等价写出：

In [25]:
df.pivot_table(index='Name',columns='Subject',values='Grade',aggfunc= lambda x: x.mean())

Subject,Chinese,Math
Name,,
San Zhang,85.0,95.0
Si Li,75.0,90.0


此外， **pivot_table 具有边际汇总的功能，可以通过设置 margins=True 来实现，其中边际的聚合方式与 aggfunc 中给出的聚合方法一致**。下面就分别统计了语文均分和数学均分、张三均分和李四均分，以及总体所有分数的均分：

In [26]:
df.pivot_table(index='Name',columns='Subject',values='Grade',aggfunc='mean',margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,85.0,95.0,90.00
Si Li,75.0,90.0,82.50
All,80.0,92.5,86.25


##### 练一练：
在上面的边际汇总例子中，行或列的汇总为新表中行元素或者列元素的平均值，而总体的汇总为新表中四个元素的平均值。这种关系一定成立吗？若不成立，请给出一个例子来说明。

- **对于 'mean' 和 'sum' 等线性聚合函数，边际汇总的关系成立。**
- **对于非线性聚合函数（如 'median'），边际汇总的关系可能不成立。**

### 3. melt——宽表变为长表
长宽表只是数据呈现方式的差异，但其包含的信息量是等价的，前面提到了利用 pivot 把长表转为宽表，那么就可以通过相应的逆操作把宽表转为长表， melt 函数就起到了这样的作用。在下面的例子中， Subject 以列索引的形式存储，现在想要将其压缩到一个列中。

In [27]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang','Si Li'],
                   'Chinese':[80,90],
                   'Math':[80,75]})

df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [28]:
# id_vars：保持不变作为行索引的列
# value_vars：需要被转换的列
# var_name：转换后列（索引）的名称
# value_name：转换后列值的变量名
df_melted = df.melt(id_vars=['Class','Name'],
                    value_vars=['Chinese','Math'],
                    var_name='Subject',
                    value_name='Grade')
df_melted


,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


melt 的主要参数和压缩的过程如下图所示：

<img src="https://inter.joyfulpandas.datawhale.club/_images/ch5_melt.png" width="80%">

前面提到了 melt 和 pivot 是一组互逆过程，那么就一定可以通过 pivot 操作把 df_melted 转回 df 的形式：

In [30]:
df_unmelted = df_melted.pivot(index=['Class','Name'],
                              columns='Subject',
                              values='Grade')
df_unmelted


,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,80
2,Si Li,90,75


In [36]:
df_unmelted1 = df_unmelted.reset_index()

# Subject	Class	Name	Chinese	Math
# 0	        1	    San Zhang	80	80
# 1	        2	    Si Li	    90	75

df_unmelted1 = df_unmelted.reset_index().rename_axis(
                             columns={'Subject':''})
df_unmelted1


#     Class	Name	Chinese	Math
# 0	    1	San Zhang	80	80
# 1	    2	Si Li	90	75

df_unmelted1.equals(df)

True

### 4. wide_to_long

**melt 方法中，在列索引中被压缩的一组值对应的列元素只能代表同一层次的含义，即 values_name**。现在如果列中包含了交叉类别，比如期中期末的类别和语文数学的类别，那么**想要把 values_name 对应的 Grade 扩充为两列分别对应语文分数和数学分数，只把期中期末的信息压缩，这种需求下就要使用 wide_to_long 函数来完成。**

In [37]:
df = pd.DataFrame({'Class':[1,2],'Name':['San Zhang','Si Li'],
                   'Chinese_Mid':[80,75],'Math_Mid':[90,95],
                   'Chinese_Final':[80,75],'Math_Final':[90,85]})
df

,Class,Name,Chinese_Mid,Math_Mid,Chinese_Final,Math_Final
0,1,San Zhang,80,90,80,90
1,2,Si Li,75,95,75,85


In [44]:
# stubnames 中的列必须在原始 DataFrame 中存在，
# 并且列名必须遵循 stubname + sep + suffix 的模式（例如 Chinese_1, Chinese_2）。
# 这里的suffix='.+') 表示后缀可以是任意字符（通常用于匹配数字，例如 1, 2, 3 等）。
# 尽管这里是数字，但默认会当成字符串
pd.wide_to_long(df,
                stubnames=['Chinese','Math'], # 要“拉长”的列的前缀名。
                i=['Class','Name'], # 保持不变的列
                j='Examination', # 新列的名称，用于表示拉长后新增的维度。j='Examination' 表示每门课程的多次考试（例如 Chinese_1, Chinese_2）将被压缩成一个名为 Examination 的列，值为 1, 2 等。
                sep='_', # 指定 stubnames 与其后缀之间的分隔符。
                suffix='.+') # 用于匹配 stubnames 后面的后缀。在这个例子中，suffix='.+' 表示后缀可以是任意字符（通常用于匹配数字，例如 1, 2, 3 等）。
                

Chinese  Math
Class Name      Examination               
1     San Zhang Mid               80    90
                Final             80    90
2     Si Li     Mid               75    95
                Final             75    85

具体的变换过程由下图进行展示，属相同概念的元素使用了一致的颜色标出：
<img src="https://inter.joyfulpandas.datawhale.club/_images/ch5_wtl.png" alt="wide_to_long" style="zoom: 80%;" />

下面给出一个**比较复杂的案例**，把之前在 pivot 一节中多列操作的结果（产生了多级索引），利用 wide_to_long 函数，将其转为原来的形态。其中，使用了第八章的 str.split 函数，目前暂时只需将其理解为对序列按照某个分隔符进行拆分即可。

In [53]:
res = pivot_multi.copy()

# 把多级列索引通过_拼接为一个单元
res.columns = res.columns.map(lambda x: '_'.join(x))
# Grade_Chinese_Mid

# 重置索引
res = res.reset_index()
res = pd.wide_to_long(res,stubnames=['Grade','rank'], # 拉长的列的前缀
                      i = ['Class','Name'], # 不变的列名
                      j = 'Subject_Exam', # 拉长后，新的列名
                      sep='_',
                      suffix='.+'
                      )
res

Grade  rank
Class Name      Subject_Exam              
1     San Zhang Chinese_Mid       80    10
                Chinese_Final     75    15
                Math_Mid          90    20
                Math_Final        85     7
2     Si Li     Chinese_Mid       85    21
                Chinese_Final     65    15
                Math_Mid          92     6
                Math_Final        88     2

In [54]:
res = res.reset_index()

res[['Subject','Examination']] = res['Subject_Exam'].str.split('_',expand=True)

res = res[['Class','Name','Examination','Subject','Grade','rank']].sort_values(by='Subject')
res

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
4,2,Si Li,Mid,Chinese,85,21
5,2,Si Li,Final,Chinese,65,15
2,1,San Zhang,Mid,Math,90,20
3,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


In [ ]:
# drop=True 删除原先的索引
res1 = res.reset_index(drop=False)

res1

,index,Class,Name,Examination,Subject,Grade,rank
0,0,1,San Zhang,Mid,Chinese,80,10
1,1,1,San Zhang,Final,Chinese,75,15
2,4,2,Si Li,Mid,Chinese,85,21
3,5,2,Si Li,Final,Chinese,65,15
4,2,1,San Zhang,Mid,Math,90,20
5,3,1,San Zhang,Final,Math,85,7
6,6,2,Si Li,Mid,Math,92,6
7,7,2,Si Li,Final,Math,88,2


In [57]:
res2 = res.reset_index(drop=True)

res2


,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2
